In [6]:
import pandas as pd

"""
USDA NASS ALTERATION NOTEBOOK
PURPOSE: 

This notebook is to be utilized for the altering of the NASS dataset
With this notebook we intend to alter the NASS format from a long format to a wide format for our purposes 

Input:
File: nass_qs_1902_to_2025.csv (raw USDA NASS Data)
Format: Long format
Source: USDA NASS QuickStats (see documentation/USDA_NASS_Data_Acquistion.md)

Output: 
Format: Wide format with 124 rows (one per year)
Columns: year, acres_harvested, acres_planted, yield_bu_per_acre, production_bushels

Authors: Brady Brooks and Aden Krueger
Date: November 2025
"""

df = pd.read_csv("../data/raw/nass_qs_1902_to_2025.csv")
print(len(df))

# filter to the stats we want with TOTAL domain
df_filtered = df[
    (df['Domain'] == 'TOTAL') &
    (df['Data Item'].isin([
        'CORN - ACRES PLANTED',
        'CORN, GRAIN - ACRES HARVESTED',
        'CORN, GRAIN - YIELD, MEASURED IN BU / ACRE',
        'CORN, GRAIN - PRODUCTION, MEASURED IN BU'
    ]))
].copy()


# clean the Value column
df_filtered['Value'] = df_filtered['Value'].astype(str).str.replace(',', '')
df_filtered['Value'] = pd.to_numeric(df_filtered['Value'], errors='coerce')


# pivot to wide format
df_wide = df_filtered.pivot_table(
    index='Year',
    columns='Data Item',
    values='Value',
    aggfunc='first'
).reset_index()

# check what columns we actually got from our alterations
print("Columns after pivoting", df_wide.columns.tolist())
print("Shape:", df_wide.shape)

# Rename using a dictionary (safer than assuming column order)
df_wide = df_wide.rename(columns={
    'CORN - ACRES PLANTED': 'acres_planted',
    'CORN, GRAIN - ACRES HARVESTED': 'acres_harvested',
    'CORN, GRAIN - YIELD, MEASURED IN BU / ACRE': 'yield_bu_per_acre',
    'CORN, GRAIN - PRODUCTION, MEASURED IN BU': 'production_bushels'
})

# rename Year to lowercase for consistency
df_wide = df_wide.rename(columns={'Year': 'year'})
print("\nFinal columns:", df_wide.columns.tolist())
len(df_wide)

print("Year range:", df_wide['year'].min(), "to", df_wide['year'].max())
print("Number of years:", df_wide['year'].max() - df_wide['year'].min() + 1)
print("Actual rows:", len(df_wide))

# Check for missing years
all_years = set(range(int(df_wide['year'].min()), int(df_wide['year'].max()) + 1))
actual_years = set(df_wide['year'].astype(int))
missing_years = sorted(all_years - actual_years)
print(f"\nMissing years: {missing_years if missing_years else 'None'}")

4641
Columns after pivoting ['Year', 'CORN - ACRES PLANTED', 'CORN, GRAIN - ACRES HARVESTED', 'CORN, GRAIN - PRODUCTION, MEASURED IN BU', 'CORN, GRAIN - YIELD, MEASURED IN BU / ACRE']
Shape: (124, 5)

Final columns: ['year', 'acres_planted', 'acres_harvested', 'production_bushels', 'yield_bu_per_acre']
Year range: 1902 to 2025
Number of years: 124
Actual rows: 124

Missing years: None


In [7]:
# sample to show our dataset
sample = df_wide.sample(n=10, random_state=42)

print("Random sample of non-NaN acres_planted values:")
sample


Random sample of non-NaN acres_planted values:


Data Item,year,acres_planted,acres_harvested,production_bushels,yield_bu_per_acre
18,1920,NaN,8425000.0,2.948750e+08,35.0
42,1944,9226000.0,8804000.0,3.997020e+08,45.4
36,1938,8513000.0,8073000.0,3.552120e+08,44.0
76,1978,11450000.0,11170000.0,1.239870e+09,111.0
53,1955,9180000.0,8885000.0,4.975600e+08,56.0
90,1992,11200000.0,11050000.0,1.646450e+09,149.0
56,1958,8664000.0,8244000.0,5.688360e+08,69.0
122,2024,10800000.0,10650000.0,2.311050e+09,217.0
121,2023,11200000.0,11050000.0,2.276300e+09,206.0
4,1906,NaN,10500000.0,4.095000e+08,39.0
